# Aim/Introduction
This notebook aims at giving a sense of all the basic operations that one can do in pandas. <br>
This are useful in day to day analysis and comes across again and again <br>
<b> Note :- </b> Deliberately, modelling realted functions are not included here. They can be kept in some seperate notebook <br>
<b> Additional scope :-</b> Many other thing, like plotting can be included in this notebook

# Notebook running guidelines
One needs to run th whole of 'Notebook requirements' first. <br> 
After that each section is made independent.

# Notebook requirements

## Imports

In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd
import datetime

## Input variables

In [2]:
data_folder = '../data/'
input_data_folder = data_folder + 'input_data/'
output_data_folder = data_folder + 'output_data/'
plots_folder = data_folder + 'plots/'

## Folder requirements
All present in the data folder provided

1. a csv file
2. a parquet file
3. multiple csv files for glob.glob
4. An excel file


# Creating DataFrame

## By Reading from file

### Reading from csv file

In [3]:
df = pd.read_csv(input_data_folder + 'Superstore_Users.csv')
print(df.shape)
df.head()

(4, 2)


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


### Reading from parquet file. 
Most S3 data will be in this format

In [4]:
df = pd.read_parquet(input_data_folder + 'Superstore_Users.parquet')
print(df.shape)
df.head()

(4, 2)


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


### Reading from excel file
Read a excel file with multiple sheets and then read one of them in a df

<b>Additional</b> :- A single sheet excel file can be read with <b>pd.read_excel</b> command
<br><b>Advice</b> :- It is advised to read from .csv file as it is faster and we are more sure of input file format

In [5]:
excel_file_read = pd.ExcelFile(input_data_folder + 'Superstore.xlsx')
print(type(excel_file_read))
print(excel_file_read.sheet_names)

<class 'pandas.io.excel.ExcelFile'>
['Orders', 'Returns', 'Users']


Now to parse a sheet from it

In [6]:
sheet_df = excel_file_read.parse('Users')
print(sheet_df.shape)
sheet_df.head()

(4, 2)


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


### Reading from clipboard. Copy paste to dataframe
<b>Activtiy Needed :- </b> Select the tabular output of last cell and copy it. Then run the next cell.<br>
<i>One can copy from excel, notepad or any text editor and run the below command to get it into df. No need to write it to csv file and then read from file<i>

In [8]:
df_copy_pasted = pd.read_clipboard()
print(df_copy_pasted.shape)
df_copy_pasted.head()

(4, 2)


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


### Reading bunch of files with a pattern
glob.glob(file_path_regrex) allows to iterate over a pattern of files

In [9]:
import glob
file_iterator = glob.glob(input_data_folder + 'Superstore_*.csv') # This is a regex (regular expression).
print(file_iterator)

['../data/input_data/Superstore_Orders.csv', '../data/input_data/Superstore_Users.csv', '../data/input_data/Superstore_Returns.csv']


#### List of dataframes by iterating
Then dataframes can be refrred by list index. <br>
One can also create a dictonary of dataframe for key based refrencing instead of list based indexing

In [10]:
df_list = []

# file_iterator coming from the previous code cell
for file in file_iterator:
    # Read file in a temp df
    df_temp = pd.read_csv(file)

    #append temp df to the list
    df_list.append(df_temp)

print('Length of list is ', len(df_list))
print('Shape of first dataframe is ', df_list[0].shape)
print('Shape of second dataframe is ', df_list[1].shape)

Length of list is  3
Shape of first dataframe is  (9426, 24)
Shape of second dataframe is  (4, 2)


## By <b>NOT</b> reading  from file

### Creating from list
Dataframe can be created from a list of list. Each list represent a row

In [11]:
df = pd.DataFrame([['col1row1', 'col2row1'], 
                   ['col1row2', 'col2row2']])
print(df.shape)
print()
print('By default columns and index are set starring from 0')
print('One can set columns and index as shown in next cell')
df

(2, 2)

By default columns and index are set starring from 0
One can set columns and index as shown in next cell


,0,1
0,col1row1,col2row1
1,col1row2,col2row2


In [12]:
# One can specify columns and index instead
df = pd.DataFrame([['col1row1', 'col2row1'], 
                   ['col1row2', 'col2row2']] , 
                  columns = ['col1', 'col2'], index=['row1', 'row2'])
print(df.shape)
df

(2, 2)


,col1,col2
row1,col1row1,col2row1
row2,col1row2,col2row2


### Creating from dict

In [13]:
data = {
    'apples': [3, 2, 0, 1], 
    'oranges': [0, 3, 7, 2]
}

#### pd.DataFrame method
It takes the keys of dictionary as columns

In [14]:
df = pd.DataFrame(data)
df

,apples,oranges
0,3,0
1,2,3
2,0,7
3,1,2


#### from_dict method of DataFrame
If you want to get the keys of dictionary as columns

In [15]:
df = pd.DataFrame.from_dict(data, orient='columns')
print(df)

print()

# Same can be achieved by from_records method
df = pd.DataFrame.from_records(data)
print(df)

   apples  oranges
0       3        0
1       2        3
2       0        7
3       1        2

   apples  oranges
0       3        0
1       2        3
2       0        7
3       1        2


#### Index
If you want to get the keys of dictionary as index rather than columns

In [16]:
df = pd.DataFrame.from_dict(data, orient='index')
print(df)

         0  1  2  3
apples   3  2  0  1
oranges  0  3  7  2


### From Json 
json can be read in multiple orientation. Understand the way you want to read the json in df first.

In [17]:
# from dict of list
df = pd.read_json('{"columns":["col 1","col 2"],"index":["row 1","row 2"], "data":[["a","b"],["c","d"]]}',
                  orient='split')
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [18]:
# from dict of dict
df = pd.read_json('{"row 1":{"col 1":"a","col 2":"b"},"row 2":{"col 1":"c","col 2":"d"}}', 
                  orient='index')
df

,col 1,col 2
row 1,a,b
row 2,c,d


In [19]:
# from list of dict
df = pd.read_json('[{"col 1":"a","col 2":"b"}, {"col 1":"c","col 2":"d"}]', 
                  orient='records')
df

,col 1,col 2
0,a,b
1,c,d


### Spliting a column of a dataframe into indiviual columns
If a column of a dataframe is in form of dictionary, it can be converted to a dataframe

In [20]:
# Creating a sample dataframe which we can split
df = pd.DataFrame( [[0, {"col 1":"a","col 2":"b"}], [1, {"col 1":"c","col 2":"d"}],
                    [2, {"col 2":"e","col 3":"f"}], [3, {"col 3":"g","col 4":"h"}]],
                 columns = ['s_no', 'metadata'])
df

,s_no,metadata
0,0,"{'col 1': 'a', 'col 2': 'b'}"
1,1,"{'col 1': 'c', 'col 2': 'd'}"
2,2,"{'col 2': 'e', 'col 3': 'f'}"
3,3,"{'col 3': 'g', 'col 4': 'h'}"


In [21]:
# Splitting is very easy. Simply apply pd.Series to the column
df['metadata'].apply(pd.Series)

,col 1,col 2,col 3,col 4
0,a,b,NaN,NaN
1,c,d,NaN,NaN
2,NaN,e,f,NaN
3,NaN,NaN,g,h


In [22]:
# If you wish to combine the new columns to original dataset, you can use this. More on concat later
new_df = pd.concat([df, df['metadata'].apply(pd.Series)], axis=1)
new_df

,s_no,metadata,col 1,col 2,col 3,col 4
0,0,"{'col 1': 'a', 'col 2': 'b'}",a,b,NaN,NaN
1,1,"{'col 1': 'c', 'col 2': 'd'}",c,d,NaN,NaN
2,2,"{'col 2': 'e', 'col 3': 'f'}",NaN,e,f,NaN
3,3,"{'col 3': 'g', 'col 4': 'h'}",NaN,NaN,g,h


# Dataframe Basic properties

In [24]:
df = pd.read_csv(input_data_folder + 'Superstore_orders.csv')

## Shape of dataframe

In [25]:
print(df.shape)

(9426, 24)


## Top and bottom n rows of dataframe
There is head and tail function to check data

In [26]:
print('By default head and tail gives 5 rows')
df.head()

By default head and tail gives 5 rows


,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [27]:
print('Check top 10 rows')
print('Try calling df.tail() to get last rows')
df.head(10)


Check top 10 rows
Try calling df.tail() to get last rows


,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523
5,23597,Medium,0.09,55.48,14.30,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-11-09,2011-11-11,1388.0523,37,2011.67,88524
6,25549,Low,0.08,120.97,26.30,3,Bonnie Potter,Delivery Truck,Corporate,Technology,...,West,Washington,Anacortes,98221,2013-07-01,2013-07-08,1001.4453,12,1451.37,88526
7,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,West,California,San Gabriel,91776,2010-12-13,2010-12-15,4390.3665,12,6362.85,90193
8,19483,Low,0.08,6.48,6.81,5,Ronnie Proctor,Regular Air,Home Office,Office Supplies,...,West,California,San Gabriel,91776,2012-05-12,2012-05-21,-141.2600,18,113.25,90197
9,24782,High,0.01,90.24,0.99,6,Dwight Hwang,Regular Air,Home Office,Office Supplies,...,West,California,San Jose,95123,2011-05-26,2011-05-26,1045.4673,16,1515.17,90194


## Index and columns. 
For easy refrence of rows and columns. <br>
Index is refrence to a row. Column names are refrence to a column <br>

In [28]:
print('Row index is numbered range')
print(df.index)
print()
print('Column index is a list')
print(df.columns)

Row index is numbered range
RangeIndex(start=0, stop=9426, step=1)

Column index is a list
Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')


## Rename columns

### Convert all column names
Lets convert the columns of table to python convention which is snake_case

In [29]:
# Fetch current column list
current_columns_list = df.columns.tolist()

new_column_list = [ '_'.join(col.lower().split(' ')) for col in current_columns_list]

print('Old Columns List')
print(current_columns_list)

print('\n New Columns List')
print(new_column_list)


Old Columns List
['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost', 'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment', 'Product Category', 'Product Sub-Category', 'Product Container', 'Product Name', 'Product Base Margin', 'Region', 'State or Province', 'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit', 'Quantity ordered new', 'Sales', 'Order ID']

 New Columns List
['row_id', 'order_priority', 'discount', 'unit_price', 'shipping_cost', 'customer_id', 'customer_name', 'ship_mode', 'customer_segment', 'product_category', 'product_sub-category', 'product_container', 'product_name', 'product_base_margin', 'region', 'state_or_province', 'city', 'postal_code', 'order_date', 'ship_date', 'profit', 'quantity_ordered_new', 'sales', 'order_id']


In [30]:
# Setting new columns is very easy
df.columns = new_column_list
df.head()

,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


### Convert a specific column name
Use .rename function. It takes in a dictionary as input. <br>
You can also use this to rename indexes. 

In [31]:
# Lets convert 'discount' to 'discount_offered'
de_renamed = df.rename(columns = {'discount':'discount_offered'})
de_renamed.head()

,row_id,order_priority,discount_offered,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


## Cell Refrencing

There are 3 methods of referncing cells in dataframe.
1. loc gets rows (or columns) with particular labels from the index.
2. iloc gets rows (or columns) at particular positions in the index (so it only takes integers).
3. ix usually tries to behave like loc but falls back to behaving like iloc if a label is not present in the index. 

<br><b>Note :- </b> ix is deprecated in current versions. So stick to loc and iloc

In [32]:
df.head()

,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,West,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523


In [33]:
print(df.loc[1,'order_priority'])

# Same thing can be achieved with iloc. 
print(df.iloc[1,1])

High
High


In [34]:
# To get the first row
df.loc[1,:] # or df.iloc[1,:]

row_id                                                           20847
order_priority                                                    High
discount                                                          0.01
unit_price                                                        2.84
shipping_cost                                                     0.93
customer_id                                                          3
customer_name                                            Bonnie Potter
ship_mode                                                  Express Air
customer_segment                                             Corporate
product_category                                       Office Supplies
product_sub-category                               Pens & Art Supplies
product_container                                             Wrap Bag
product_name            SANFORD Liquid Accent™ Tank-Style Highlighters
product_base_margin                                               0.54
region

In [35]:
# To get the first columns
df.loc[:,'row_id'].head() # or df.iloc[:,1]

0    18606
1    20847
2    23086
3    23087
4    23088
Name: row_id, dtype: int64

In [36]:
# The upper-left 3 X 3 entries
df.iloc[:3, :3] 

,row_id,order_priority,discount
0,18606,Not Specified,0.01
1,20847,High,0.01
2,23086,Not Specified,0.03


<b>Additional :- </b> Sometimes one need the location of partiular column. That can be done as below

In [37]:
# Get location of index
df.columns.get_loc('shipping_cost')

4

In [38]:
# Then you can call iloc as
df.iloc[2, df.columns.get_loc('shipping_cost')]

6.15

## Column Filtering and Rows Subsetting

### Column subset

In [39]:
# Just define a column list you want and then take list like this
df_subset = df[['row_id', 'customer_name', 'customer_id', 'order_date']]
print(df_subset.shape)
df_subset.head()

(9426, 4)


,row_id,customer_name,customer_id,order_date
0,18606,Janice Fletcher,2,2012-05-28
1,20847,Bonnie Potter,3,2010-07-07
2,23086,Bonnie Potter,3,2011-07-27
3,23087,Bonnie Potter,3,2011-07-27
4,23088,Bonnie Potter,3,2011-07-27


### Rows filtering

In [40]:
df_customer_id_filter = df[df['customer_id'] == 5]
print(df_customer_id_filter.shape)
df_customer_id_filter.head()

(2, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
7,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,West,California,San Gabriel,91776,2010-12-13,2010-12-15,4390.3665,12,6362.85,90193
8,19483,Low,0.08,6.48,6.81,5,Ronnie Proctor,Regular Air,Home Office,Office Supplies,...,West,California,San Gabriel,91776,2012-05-12,2012-05-21,-141.2600,18,113.25,90197


In [41]:
# You can combine them both
df_customer_id_filter_2 = df[df['customer_id'] == 5][['row_id', 'customer_name', 'customer_id', 'order_date']]
print(df_customer_id_filter_2.shape)
df_customer_id_filter_2.head()

(2, 4)


,row_id,customer_name,customer_id,order_date
7,20228,Ronnie Proctor,5,2010-12-13
8,19483,Ronnie Proctor,5,2012-05-12


### Filtering using loc command
use df.loc[row_condition, column_to_subset]

df['customer_id'] == 5 gives a series of True and False. <br>
Once this condition is given to df, it picks only the true rows from that and leaves false ones

In [42]:
df_customer_id_filter_3 = df.loc[df['customer_id'] == 5, ['row_id', 'customer_name', 'customer_id', 'order_date']]
print(df_customer_id_filter_3.shape)
df_customer_id_filter_3.head()


(2, 4)


,row_id,customer_name,customer_id,order_date
7,20228,Ronnie Proctor,5,2010-12-13
8,19483,Ronnie Proctor,5,2012-05-12


## Set and reset index

Notice that in df_customer_id_filter_3, index does not start from 0. It is 7 and 8 <br>
This is beacuse this is just a slice of original df with original index retained

In [43]:
# Let's reset the index to start from 0. This would create a new column index with value 7,8
df_customer_id_filter_3.reset_index()

,index,row_id,customer_name,customer_id,order_date
0,7,20228,Ronnie Proctor,5,2010-12-13
1,8,19483,Ronnie Proctor,5,2012-05-12


In [44]:
# Let's reset the index to start from 0 but without creating the new column of index
df_customer_id_filter_3.reset_index(drop=True)

,row_id,customer_name,customer_id,order_date
0,20228,Ronnie Proctor,5,2010-12-13
1,19483,Ronnie Proctor,5,2012-05-12


In [45]:
# Suppose we want to set some other column as index. Let's say row_id. Let's do that
df_set_index = df_customer_id_filter_3.set_index('row_id')
df_set_index

,customer_name,customer_id,order_date
row_id,,,
20228,Ronnie Proctor,5,2010-12-13
19483,Ronnie Proctor,5,2012-05-12


In [46]:
print(df.index)
print(df_set_index.index)

RangeIndex(start=0, stop=9426, step=1)
Int64Index([20228, 19483], dtype='int64', name='row_id')


<b>Notice</b> the diffence between index of df and df_set_index. <br>
index of df is unnamed while index of df_set_index has a name 'row_id'. <br>
One can set/change/delete the name of index of a df <br>
Similary, column can also have a name which can be set/changed/deleted


In [47]:
df_set_index.rename_axis('my_index')

,customer_name,customer_id,order_date
my_index,,,
20228,Ronnie Proctor,5,2010-12-13
19483,Ronnie Proctor,5,2012-05-12


In [48]:
# Rename axis and reset_index will result in below
df_set_index.rename_axis('my_index').reset_index()

,my_index,customer_name,customer_id,order_date
0,20228,Ronnie Proctor,5,2010-12-13
1,19483,Ronnie Proctor,5,2012-05-12


In [49]:
# Setting name of column. Renaming from blank
df_set_index.rename_axis('my_column', axis=1)

my_column,customer_name,customer_id,order_date
row_id,,,
20228,Ronnie Proctor,5,2010-12-13
19483,Ronnie Proctor,5,2012-05-12


In [50]:
# One can check index and column name with below command
print(df_set_index.index.name)
print(df_set_index.columns.name)

row_id
None


<b>NOTE :- </b> .rename, .reset_index(), .set_index(), .rename_axis() all return a new df. Does not update existing df <br>
To update the existing dataframe we can specify a parameter inplace=True

# Data Summary and Statistics
These are to report the statistics. Same can be visulized using plots. Which we can see later 

## Numerical columns 
In numerical columns, we mostly need to see what is distibution of the field as in its mean, median, percentiles etc

In [3]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [4]:
# Gives you basic stats for all numerical columns
df.describe()

,row_id,discount,unit_price,shipping_cost,customer_id,product_base_margin,postal_code,profit,quantity_ordered_new,sales,order_id
count,9426.000000,9426.000000,9426.000000,9426.000000,9426.000000,9354.000000,9426.000000,9426.000000,9426.000000,9426.000000,9426.000000
mean,20241.015277,0.049628,88.303686,12.795142,1738.422236,0.512189,52446.327286,139.236410,13.798430,949.706272,82318.489073
std,6101.890965,0.031798,281.540982,17.181203,979.167197,0.135229,29374.597802,998.486483,15.107688,2598.019818,19149.448857
min,2.000000,0.000000,0.990000,0.490000,2.000000,0.350000,1001.000000,-16476.838000,1.000000,1.320000,6.000000
25%,19330.250000,0.020000,6.480000,3.192500,898.000000,0.380000,29406.000000,-74.017375,5.000000,61.282500,86737.250000
50%,21686.500000,0.050000,20.990000,6.050000,1750.000000,0.520000,52302.000000,2.567600,10.000000,203.455000,88344.500000
75%,24042.750000,0.080000,85.990000,13.990000,2578.750000,0.590000,78516.000000,140.243850,17.000000,776.402500,89987.750000
max,26399.000000,0.250000,6783.020000,164.730000,3403.000000,0.850000,99362.000000,16332.414000,170.000000,100119.160000,91591.000000


In [5]:
# To get specific distribution values
df.describe(percentiles=[0.9,0.99])

,row_id,discount,unit_price,shipping_cost,customer_id,product_base_margin,postal_code,profit,quantity_ordered_new,sales,order_id
count,9426.000000,9426.000000,9426.000000,9426.000000,9426.000000,9354.000000,9426.000000,9426.000000,9426.000000,9426.000000,9426.000000
mean,20241.015277,0.049628,88.303686,12.795142,1738.422236,0.512189,52446.327286,139.236410,13.798430,949.706272,82318.489073
std,6101.890965,0.031798,281.540982,17.181203,979.167197,0.135229,29374.597802,998.486483,15.107688,2598.019818,19149.448857
min,2.000000,0.000000,0.990000,0.490000,2.000000,0.350000,1001.000000,-16476.838000,1.000000,1.320000,6.000000
50%,21686.500000,0.050000,20.990000,6.050000,1750.000000,0.520000,52302.000000,2.567600,10.000000,203.455000,88344.500000
90%,25456.500000,0.090000,204.100000,35.000000,3078.000000,0.720000,92672.000000,689.753700,26.000000,2323.645000,90925.000000
99%,26304.750000,0.100000,810.980000,70.200000,3374.000000,0.830000,98387.000000,4387.316700,81.000000,10385.720000,91524.750000
max,26399.000000,0.250000,6783.020000,164.730000,3403.000000,0.850000,99362.000000,16332.414000,170.000000,100119.160000,91591.000000


In [6]:
# Stat for one column. If need to be used in some filtering or else
print('Mean =', df['discount'].mean())
print('Std =', df['discount'].std())
print('Sum =', df['discount'].sum())
print('Median =', df['discount'].median())
print('90 Percentile =', df['discount'].quantile(0.90))

Mean = 0.049627625716102734
Std = 0.03179842507529814
Sum = 467.7900000000001
Median = 0.05
90 Percentile = 0.09


## Categorical columns
In categorical_column, we mostly see how many times different values of field comes up

In [7]:
df['customer_segment'].value_counts()

Corporate         3375
Home Office       2316
Consumer          1894
Small Business    1841
Name: customer_segment, dtype: int64

In [8]:
df['customer_segment'].value_counts(normalize=True)

Corporate         0.358052
Home Office       0.245703
Consumer          0.200934
Small Business    0.195311
Name: customer_segment, dtype: float64

In [9]:
# To get both in one go, I have written a simple function
def get_value_counts(series):
    return pd.concat([series.value_counts().rename('count'), 
                      series.value_counts(normalize=True).rename('percentage')], axis=1)


In [10]:
get_value_counts(df['customer_segment'])

,count,percentage
Corporate,3375,0.358052
Home Office,2316,0.245703
Consumer,1894,0.200934
Small Business,1841,0.195311


# New column creation

In [59]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [60]:
# just making a copy of df
df_1 = df.copy()

## Making a constant column

In [61]:
df_1['constant'] = 'c'
df_1.head()

,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id,constant
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525,c
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522,c
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523,c
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523,c
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523,c


## Making a column from list. 

In [62]:
# Give the list with lenght same as the length of df. 
df_1['s_no'] = list(range(len(df_1)))
df_1.head()

,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id,constant,s_no
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525,c,0
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522,c,1
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523,c,2
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523,c,3
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523,c,4


## Inserting column in front

In [63]:
df_1.insert(0,'s_no_front', list(range(len(df_1))))
df_1.head()

,s_no_front,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,...,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id,constant,s_no
0,0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,...,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525,c,0
1,1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,...,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522,c,1
2,2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,...,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523,c,2
3,3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,...,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523,c,3
4,4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,...,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523,c,4


## Creating column conditional on 1 column of df

### Based on Filter Flag

In [64]:
# Create column to show if discount was greater than 0.05
df_1['discount_gt_1'] = df_1['discount'] > 0.05
df_1[['discount', 'discount_gt_1']].tail()

,discount,discount_gt_1
9421,0.06,True
9422,0.00,False
9423,0.08,True
9424,0.10,True
9425,0.09,True


### Using .apply method. If else in it
Same thing can be acheived using .apply function.<br>
Advantage over above method is that we can give the columns value as we like instead of True/False

In [65]:
df_1['discount_gt_2'] = df_1['discount'].apply(lambda x: 'High Discount' if x > 0.05 else 'Low Discount')
df_1[['discount', 'discount_gt_1', 'discount_gt_2']].tail()

,discount,discount_gt_1,discount_gt_2
9421,0.06,True,High Discount
9422,0.00,False,Low Discount
9423,0.08,True,High Discount
9424,0.10,True,High Discount
9425,0.09,True,High Discount


### Using .apply operation with lambda function
Advantage over above 1 is that we can create complicate logic instead of simple if else

In [66]:
def get_discount_category(discount_value):
    if discount_value <= 0:
        return 'No Discount'
    elif discount_value < 0.07:
        return 'Low Discount'
    elif discount_value < 0.10:
        return 'Mid Discount'
    else:
        return 'High Discount'

In [67]:
df_1['discount_gt_3'] = df_1['discount'].apply(lambda x: get_discount_category(x))
df_1[['discount', 'discount_gt_1', 'discount_gt_2', 'discount_gt_3']].tail()

,discount,discount_gt_1,discount_gt_2,discount_gt_3
9421,0.06,True,High Discount,Low Discount
9422,0.00,False,Low Discount,No Discount
9423,0.08,True,High Discount,Mid Discount
9424,0.10,True,High Discount,High Discount
9425,0.09,True,High Discount,Mid Discount


### Using .apply without lambda operator. 
The value of the series directly gets passed in the function as argument

In [68]:
# Same as above can be acheived with this below
df_1['discount_gt_4'] = df_1['discount'].apply(get_discount_category)
df_1[['discount', 'discount_gt_1', 'discount_gt_2', 'discount_gt_3', 'discount_gt_4']].tail()

,discount,discount_gt_1,discount_gt_2,discount_gt_3,discount_gt_4
9421,0.06,True,High Discount,Low Discount,Low Discount
9422,0.00,False,Low Discount,No Discount,No Discount
9423,0.08,True,High Discount,Mid Discount,Mid Discount
9424,0.10,True,High Discount,High Discount,High Discount
9425,0.09,True,High Discount,Mid Discount,Mid Discount


### Using .apply without lambda operator. With additional constant arguments of function

In [69]:
# The function now takes a additional input which is added to all conditional limits
def get_discount_category_v2(discount_value, zero_at):
    if discount_value <= zero_at + 0:
        return 'No Discount'
    elif discount_value < zero_at + 0.07:
        return 'Low Discount'
    elif discount_value < zero_at + 0.10:
        return 'Mid Discount'
    else:
        return 'High Discount'

In [70]:
# Same can be acheived 
df_1['discount_gt_5'] = df_1['discount'].apply(get_discount_category_v2, zero_at=0.01)
df_1[['discount', 'discount_gt_1', 'discount_gt_2', 'discount_gt_3', 'discount_gt_4',
     'discount_gt_5']].tail()

,discount,discount_gt_1,discount_gt_2,discount_gt_3,discount_gt_4,discount_gt_5
9421,0.06,True,High Discount,Low Discount,Low Discount,Low Discount
9422,0.00,False,Low Discount,No Discount,No Discount,No Discount
9423,0.08,True,High Discount,Mid Discount,Mid Discount,Mid Discount
9424,0.10,True,High Discount,High Discount,High Discount,Mid Discount
9425,0.09,True,High Discount,Mid Discount,Mid Discount,Mid Discount


### With list map lambda
Advantage is that we can pass a different dataframe as input to the function. Length should be same

Notice that df and df.set_index('row_id') are different dataframes as index are different and number of columns are also different

In [71]:
other_df = df.set_index('row_id')
other_df.head(2)

,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,product_sub-category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
row_id,,,,,,,,,,,,,,,,,,,,,
18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [72]:
df_1['discount_gt_6'] = list(map(lambda x: get_discount_category_v2(x,0.01), 
                                 other_df['discount']))
df_1[['discount', 'discount_gt_1', 'discount_gt_2', 'discount_gt_3', 
      'discount_gt_4', 'discount_gt_5', 'discount_gt_6']].tail()

,discount,discount_gt_1,discount_gt_2,discount_gt_3,discount_gt_4,discount_gt_5,discount_gt_6
9421,0.06,True,High Discount,Low Discount,Low Discount,Low Discount,Low Discount
9422,0.00,False,Low Discount,No Discount,No Discount,No Discount,No Discount
9423,0.08,True,High Discount,Mid Discount,Mid Discount,Mid Discount,Mid Discount
9424,0.10,True,High Discount,High Discount,High Discount,Mid Discount,Mid Discount
9425,0.09,True,High Discount,Mid Discount,Mid Discount,Mid Discount,Mid Discount


### apply with passing the full row.
Inside function, we will need to tell which column to apply logic on. <br>
Useful if a lot of columns are used in the logic. Not much useful in single column dependency <br>
Little slower than earlier methods. <br>
<b>Note :- </b> Needs to be told which axis to look at. By default function is applied to columns not rows.

In [73]:
# The function now takes a additional input which is added to all conditional limits
def get_discount_category_v3(row, zero_at):
    if row['discount'] <= zero_at + 0:
        return 'No Discount'
    elif row['discount'] < zero_at + 0.07:
        return 'Low Discount'
    elif row['discount'] < zero_at + 0.10:
        return 'Mid Discount'
    else:
        return 'High Discount'

In [74]:
# Same can be acheived 
df_1['discount_gt_7'] = df_1.apply(lambda row: get_discount_category_v3(row,0.01), axis=1)
df_1[['discount', 'discount_gt_1', 'discount_gt_2', 'discount_gt_3', 
      'discount_gt_4', 'discount_gt_5', 'discount_gt_6', 'discount_gt_7']].tail()

,discount,discount_gt_1,discount_gt_2,discount_gt_3,discount_gt_4,discount_gt_5,discount_gt_6,discount_gt_7
9421,0.06,True,High Discount,Low Discount,Low Discount,Low Discount,Low Discount,Low Discount
9422,0.00,False,Low Discount,No Discount,No Discount,No Discount,No Discount,No Discount
9423,0.08,True,High Discount,Mid Discount,Mid Discount,Mid Discount,Mid Discount,Mid Discount
9424,0.10,True,High Discount,High Discount,High Discount,Mid Discount,Mid Discount,Mid Discount
9425,0.09,True,High Discount,Mid Discount,Mid Discount,Mid Discount,Mid Discount,Mid Discount


## Creating column conditional on more than 1 column of df

### Filter Based

In [75]:
# Create column to show if discount was greater than 0.05 and order_priority was also high
df_1['discount_gt_v2_1'] = (df_1['discount'] > 0.05) & (df_1['order_priority'] == 'High')
df_1[['discount', 'discount_gt_v2_1']].tail()

,discount,discount_gt_v2_1
9421,0.06,False
9422,0.00,False
9423,0.08,False
9424,0.10,True
9425,0.09,False


### .apply lambda with row. 


In [76]:
# Function can be created and row and can be passed if one likes. Skipping that example

df_1['discount_gt_v2_2'] = df_1[['discount', 'order_priority']].apply(lambda row: 
                                                                      1 if 
                                                                      (row['discount'] > 0.05 and 
                                                                       row['order_priority'] == 'High')
                                                                      else 0, 
                                                                      axis=1)
df_1[['discount', 'discount_gt_v2_1', 'discount_gt_v2_2']].tail()


,discount,discount_gt_v2_1,discount_gt_v2_2
9421,0.06,False,0
9422,0.00,False,0
9423,0.08,False,0
9424,0.10,True,1
9425,0.09,False,0


### List map lambda
Notice the use of 2 different df's in the right hand side

In [77]:
df_1['discount_gt_v2_3'] = list(map(lambda x, y: 1 if (x > 0.05 and y == 'High') else 0, 
                                   other_df['discount'], df_1['order_priority']))

df_1[['discount', 'discount_gt_v2_1', 'discount_gt_v2_2', 'discount_gt_v2_3']].tail()

,discount,discount_gt_v2_1,discount_gt_v2_2,discount_gt_v2_3
9421,0.06,False,0,0
9422,0.00,False,0,0
9423,0.08,False,0,0
9424,0.10,True,1,1
9425,0.09,False,0,0


## Binning Column

### Into equal range bins

In [78]:
df_1['quantity_ordered_new'].describe()

count    9426.000000
mean       13.798430
std        15.107688
min         1.000000
25%         5.000000
50%        10.000000
75%        17.000000
max       170.000000
Name: quantity_ordered_new, dtype: float64

In [79]:
df_1['quantity_ordered_new_equal_bins_1'] = pd.cut(df_1['quantity_ordered_new'], bins=5)
df_1['quantity_ordered_new_equal_bins_1'].value_counts()

(0.831, 34.8]     8737
(34.8, 68.6]       538
(68.6, 102.4]      125
(136.2, 170.0]      16
(102.4, 136.2]      10
Name: quantity_ordered_new_equal_bins_1, dtype: int64

Specify lables to the new bins created

In [80]:
df_1['quantity_ordered_new_equal_bins_2'] = pd.cut(df_1['quantity_ordered_new'], bins=5,
                                              labels = [0,1,2,3,4])
df_1['quantity_ordered_new_equal_bins_2'].value_counts()

0    8737
1     538
2     125
4      16
3      10
Name: quantity_ordered_new_equal_bins_2, dtype: int64

### Into Bins of our choice

In [81]:
df_1['quantity_ordered_new_our_bins_1'] = pd.cut(df_1['quantity_ordered_new'], 
                                             bins = [0,20, 50, 80, 100, 200])
df_1['quantity_ordered_new_our_bins_1'].value_counts()

(0, 20]       7861
(20, 50]      1254
(50, 80]       213
(80, 100]       72
(100, 200]      26
Name: quantity_ordered_new_our_bins_1, dtype: int64

With left boundary included and right excluded

In [82]:
df_1['quantity_ordered_new_our_bins_2'] = pd.cut(df_1['quantity_ordered_new'], 
                                             bins = [0,20, 50, 80, 100, 200], right=False)
df_1['quantity_ordered_new_our_bins_2'].value_counts()

[0, 20)       7683
[20, 50)      1423
[50, 80)       222
[80, 100)       70
[100, 200)      28
Name: quantity_ordered_new_our_bins_2, dtype: int64

### Equal quantity bins
Will find the quantile values and then split the column into bins. <br>
The # of points may bot be exactly equal due to boundary point inclusions

In [83]:
df_1['quantity_ordered_new_equal_quantity_1'] = pd.qcut(df_1['quantity_ordered_new'], q=4)
df_1['quantity_ordered_new_equal_quantity_1'].value_counts()

(0.999, 5.0]     2547
(10.0, 17.0]     2419
(5.0, 10.0]      2314
(17.0, 170.0]    2146
Name: quantity_ordered_new_equal_quantity_1, dtype: int64

## One hot encoding
pd.get_dummies converts a variable to dummy vairables. Later it can be concatted to original df if wanted

In [84]:
pd.get_dummies(df['customer_segment'], prefix='Customer_segment', prefix_sep='|').head()

,Customer_segment|Consumer,Customer_segment|Corporate,Customer_segment|Home Office,Customer_segment|Small Business
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


## Spliting a metadeta column of a dataframe into indiviual columns

In [ ]:
# Creating a sample dataframe which we can split
df = pd.DataFrame( [[0, {"col 1":"a","col 2":"b"}], [1, {"col 1":"c","col 2":"d"}],
                    [2, {"col 2":"e","col 3":"f"}], [3, {"col 3":"g","col 4":"h"}]],
                 columns = ['s_no', 'metadata'])
df

In [23]:
df['metadata'].apply(pd.Series)

,col 1,col 2,col 3,col 4
0,a,b,NaN,NaN
1,c,d,NaN,NaN
2,NaN,e,f,NaN
3,NaN,NaN,g,h


# Grouping and Aggregation

In [85]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


## Groupby Operation
Understanding what group by does

In [86]:
grouped_object = df.groupby('ship_mode')
type(grouped_object)

pandas.core.groupby.generic.DataFrameGroupBy

In [87]:
for grouped_key, grouped_df in grouped_object:
    print('Grouped by key =', grouped_key+'.', '  Shape of grouped df =', grouped_df.shape)

Grouped by key = Delivery Truck.   Shape of grouped df = (1283, 24)
Grouped by key = Express Air.   Shape of grouped df = (1107, 24)
Grouped by key = Regular Air.   Shape of grouped df = (7036, 24)


In [88]:
df['ship_mode'].value_counts()

Regular Air       7036
Delivery Truck    1283
Express Air       1107
Name: ship_mode, dtype: int64

So, one can create a group by object to create dataframes with that value and loop over those dataframes to fetch what is needed <br>
But groupby give multiple other functions to avoid looping through this dataframes

### Substitue for value_counts
It is much helpful if we groupby multiple columns instead of single

In [89]:
df.groupby('ship_mode').size()

ship_mode
Delivery Truck    1283
Express Air       1107
Regular Air       7036
dtype: int64

In [90]:
df.groupby(['ship_mode', 'region']).size()

ship_mode       region 
Delivery Truck  Central     386
                East        329
                South       242
                West        326
Express Air     Central     322
                East        282
                South       241
                West        262
Regular Air     Central    2191
                East       1678
                South      1471
                West       1696
dtype: int64

You can combine groupby and value counts too

In [91]:
# region wise % split of ship_mode
df.groupby('ship_mode')['region'].value_counts(normalize=True)

ship_mode       region 
Delivery Truck  Central    0.300857
                East       0.256430
                West       0.254092
                South      0.188620
Express Air     Central    0.290876
                East       0.254743
                West       0.236676
                South      0.217706
Regular Air     Central    0.311399
                West       0.241046
                East       0.238488
                South      0.209068
Name: region, dtype: float64

Or even use custom function that you might have defined.

In [92]:
get_value_counts(df.groupby('ship_mode')['region'])

count  percentage
ship_mode      region                    
Delivery Truck Central    386    0.300857
               East       329    0.256430
               West       326    0.254092
               South      242    0.188620
Express Air    Central    322    0.290876
               East       282    0.254743
               West       262    0.236676
               South      241    0.217706
Regular Air    Central   2191    0.311399
               West      1696    0.241046
               East      1678    0.238488
               South     1471    0.209068

<b> Notice :- </b> That the key by which you group becomes a named index for the df

In [93]:
df.groupby('ship_mode').size().index

Index(['Delivery Truck', 'Express Air', 'Regular Air'], dtype='object', name='ship_mode')

### Group by a column and stats of other column(s)

In [94]:
df.groupby('ship_mode')['profit'].mean()
# Similarly, you can use .std(), .sum(), .max(), .min() and many other functions

ship_mode
Delivery Truck    208.933678
Express Air       136.204485
Regular Air       127.004282
Name: profit, dtype: float64

You can also run the describe function instead of finding every stats seprately

In [95]:
df.groupby('ship_mode')['profit'].describe()

,count,mean,std,min,25%,50%,75%,max
ship_mode,,,,,,,,
Delivery Truck,1283.0,208.933678,1687.365884,-13706.4640,-356.2785,-53.78400,458.599800,16332.4140
Express Air,1107.0,136.204485,708.666816,-5972.5239,-47.8340,7.67104,150.437250,7690.6089
Regular Air,7036.0,127.004282,858.391298,-16476.8380,-56.8445,3.32400,117.294125,12504.9045


Can be used for aggregating multiple columns too

In [96]:
df.groupby('ship_mode')[['profit', 'shipping_cost']].mean()

,profit,shipping_cost
ship_mode,,
Delivery Truck,208.933678,45.197171
Express Air,136.204485,8.039169
Regular Air,127.004282,7.634974


### .agg operator
Advantage of this method is that we can give multiple aggeration methods for a column <br>
And different agreagation methods for different columns

In [97]:
df.groupby('ship_mode').agg({'profit': 'mean'})

,profit
ship_mode,
Delivery Truck,208.933678
Express Air,136.204485
Regular Air,127.004282


In [98]:
df.groupby('ship_mode').agg({'profit': ['mean', 'std']})

profit             
                      mean          std
ship_mode                              
Delivery Truck  208.933678  1687.365884
Express Air     136.204485   708.666816
Regular Air     127.004282   858.391298

In [100]:
agg_df_1 = df.groupby(['ship_mode','region']).agg({'profit': ['mean', 'std'], 
                                                   'shipping_cost': 'sum'})
agg_df_1

profit              shipping_cost
                              mean          std           sum
ship_mode      region                                        
Delivery Truck Central  228.070616  1681.055421      17907.50
               East     234.455276  1474.245757      14811.35
               South    141.964085  1354.429931      10606.97
               West     210.231767  2079.717883      14662.15
Express Air    Central  167.144535   650.868645       2605.58
               East     139.729189   698.265667       2513.71
               South     65.838272   911.364207       1841.98
               West     159.111334   558.686679       1938.09
Regular Air    Central  172.510164   770.697203      16961.44
               East     155.558265   749.403290      13068.45
               South     36.695350  1026.821585      10882.38
               West     118.294061   900.042335      12807.41

#### Multi Level Indexing
In group by and aggreagation, you will see a lot of multi level indexing. And it may get irritation dealing with it <br> 
So let's have a look at that once

Let's look at this agg_df_1 for a moment

In [101]:
agg_df_1.index

MultiIndex(levels=[['Delivery Truck', 'Express Air', 'Regular Air'], ['Central', 'East', 'South', 'West']],
           codes=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]],
           names=['ship_mode', 'region'])

In [102]:
agg_df_1.columns

MultiIndex(levels=[['profit', 'shipping_cost'], ['mean', 'std', 'sum']],
           codes=[[0, 0, 1], [0, 1, 2]])

Both columns and index and multilevel. <br>
Basically, in a dataframe there can be multiple level of index. As we have seen and realized the need here

<b> Cell Refrencing </b> will work in the same way <br>
Just the single index will now be replaced by tuples

In [103]:
agg_df_1.loc[('Delivery Truck', 'East'), :]

profit         mean      234.455276
               std      1474.245757
shipping_cost  sum     14811.350000
Name: (Delivery Truck, East), dtype: float64

In [104]:
agg_df_1.loc[:, ('profit', 'mean')]

ship_mode       region 
Delivery Truck  Central    228.070616
                East       234.455276
                South      141.964085
                West       210.231767
Express Air     Central    167.144535
                East       139.729189
                South       65.838272
                West       159.111334
Regular Air     Central    172.510164
                East       155.558265
                South       36.695350
                West       118.294061
Name: (profit, mean), dtype: float64

Look at the columns of agg_df_1 now. <br>
We will try to bring this in a single level column now without losing the meaning of the columns

In [105]:
agg_df_1.columns

MultiIndex(levels=[['profit', 'shipping_cost'], ['mean', 'std', 'sum']],
           codes=[[0, 0, 1], [0, 1, 2]])

In [106]:
# df.columns.values gives an array of the multiindex columns with the values that it takes
agg_df_1.columns.values

array([('profit', 'mean'), ('profit', 'std'), ('shipping_cost', 'sum')],
      dtype=object)

In [107]:
['_'.join(col).strip() for col in agg_df_1.columns.values]

['profit_mean', 'profit_std', 'shipping_cost_sum']

In [108]:
agg_df_1.columns = ['_'.join(col).strip() for col in agg_df_1.columns.values]
agg_df_1

profit_mean   profit_std  shipping_cost_sum
ship_mode      region                                              
Delivery Truck Central   228.070616  1681.055421           17907.50
               East      234.455276  1474.245757           14811.35
               South     141.964085  1354.429931           10606.97
               West      210.231767  2079.717883           14662.15
Express Air    Central   167.144535   650.868645            2605.58
               East      139.729189   698.265667            2513.71
               South      65.838272   911.364207            1841.98
               West      159.111334   558.686679            1938.09
Regular Air    Central   172.510164   770.697203           16961.44
               East      155.558265   749.403290           13068.45
               South      36.695350  1026.821585           10882.38
               West      118.294061   900.042335           12807.41

Fixing the mutliindex for rows is easy. We just have to do reset_index()

In [109]:
# .reset_index() brings the index back to the dataframe and makes it similar to format we are used to
agg_df_1.reset_index()

,ship_mode,region,profit_mean,profit_std,shipping_cost_sum
0,Delivery Truck,Central,228.070616,1681.055421,17907.50
1,Delivery Truck,East,234.455276,1474.245757,14811.35
2,Delivery Truck,South,141.964085,1354.429931,10606.97
3,Delivery Truck,West,210.231767,2079.717883,14662.15
4,Express Air,Central,167.144535,650.868645,2605.58
5,Express Air,East,139.729189,698.265667,2513.71
6,Express Air,South,65.838272,911.364207,1841.98
7,Express Air,West,159.111334,558.686679,1938.09
8,Regular Air,Central,172.510164,770.697203,16961.44
9,Regular Air,East,155.558265,749.403290,13068.45


### custom functions in aggregators
We can define our own aggregation functions and apply them

In [110]:
df['product_category'].value_counts()

Office Supplies    5181
Technology         2312
Furniture          1933
Name: product_category, dtype: int64

Let's say we want to find out count of non Technology product_category for a particular region

In [111]:
# Define you function assuming 'x' is a list.
def count_of_non_tech_function(x):
    count = 0
    for i in x:
        if i != 'Technology':
            count += 1
    return count

In [112]:
count_of_non_tech_lambda = lambda x: count_of_non_tech_function(x)

In [113]:
df.groupby('region').agg({'product_category': count_of_non_tech_lambda})

,product_category
region,
Central,2182
East,1751
South,1469
West,1712


We can verify this number by applying group by on filtered dataset instead of full dataset. Like below

In [114]:
df[df['product_category'] != 'Technology'].groupby('region').size()

region
Central    2182
East       1751
South      1469
West       1712
dtype: int64

### Aggregation function for strings columns

In [115]:
df.groupby('customer_name').size().sort_values(ascending=False).head()

customer_name
Louis Parrish            27
Jenny Gold               26
Leigh Burnette Hurley    22
Sean N Boyer             22
Glen Caldwell            21
dtype: int64

Let's see for each customer, how many distinct city order came from <br>
We will also calculate total orders from that customer

In [116]:
# Group by, Then reset Index, Then sort by the ditinct count of city. Then take top 5
df.groupby('customer_name').agg({'city': lambda x: len(set(x))}).reset_index().sort_values(by='city', 
                                                                                           ascending=False).head()


,customer_name,city
0,Aaron Davies Bruce,1
1805,Marvin Yang,1
1797,Marvin MacDonald,1
1798,Marvin Parrott,1
1799,Marvin Patrick,1


As top value is 1, we can conclude that every customer is ordering from 1 city only.

Let's check how many distinct types of product is the customer ordering

In [117]:
df.groupby('customer_name').agg({'product_category': lambda x: len(set(x))}).reset_index().head()


,customer_name,product_category
0,Aaron Davies Bruce,2
1,Aaron Day,2
2,Aaron Dillon,2
3,Aaron Fuller Davidson,1
4,Aaron Riggs,3


So there are few customers who order all 3 types of product and there are some who order only 1 type

We can also check what these catorgories for each of them are

In [118]:
df.groupby('customer_name').agg({'product_category':lambda x: set(x)}).head()

,product_category
customer_name,
Aaron Davies Bruce,"{Office Supplies, Furniture}"
Aaron Day,"{Office Supplies, Technology}"
Aaron Dillon,"{Office Supplies, Furniture}"
Aaron Fuller Davidson,{Furniture}
Aaron Riggs,"{Office Supplies, Furniture, Technology}"


<b>Advise :- </b> 
1. Loop over grouped obejct only when you want to apply operations over filtered df for a column. For any aggregation, use .agg method only
2. Create a agg_dict before group by function and pass this dict to .agg functions.
3. Look to reset index and change column names
4. Avoid writing custom functions which go through every element. Will make things slow.
5. If a column is repeating for the group, and any rows value will suffice, then take 'max'/'min' of that column in aggregate

# Transpose, Pivot and Reshape

In [11]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


To bring index to columns and vice-versa, use df.T

In [12]:
df.describe()

,row_id,discount,unit_price,shipping_cost,customer_id,product_base_margin,postal_code,profit,quantity_ordered_new,sales,order_id
count,9426.000000,9426.000000,9426.000000,9426.000000,9426.000000,9354.000000,9426.000000,9426.000000,9426.000000,9426.000000,9426.000000
mean,20241.015277,0.049628,88.303686,12.795142,1738.422236,0.512189,52446.327286,139.236410,13.798430,949.706272,82318.489073
std,6101.890965,0.031798,281.540982,17.181203,979.167197,0.135229,29374.597802,998.486483,15.107688,2598.019818,19149.448857
min,2.000000,0.000000,0.990000,0.490000,2.000000,0.350000,1001.000000,-16476.838000,1.000000,1.320000,6.000000
25%,19330.250000,0.020000,6.480000,3.192500,898.000000,0.380000,29406.000000,-74.017375,5.000000,61.282500,86737.250000
50%,21686.500000,0.050000,20.990000,6.050000,1750.000000,0.520000,52302.000000,2.567600,10.000000,203.455000,88344.500000
75%,24042.750000,0.080000,85.990000,13.990000,2578.750000,0.590000,78516.000000,140.243850,17.000000,776.402500,89987.750000
max,26399.000000,0.250000,6783.020000,164.730000,3403.000000,0.850000,99362.000000,16332.414000,170.000000,100119.160000,91591.000000


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
row_id,9426.0,20241.015277,6101.890965,2.000,19330.250000,21686.5000,24042.75000,26399.000
discount,9426.0,0.049628,0.031798,0.000,0.020000,0.0500,0.08000,0.250
unit_price,9426.0,88.303686,281.540982,0.990,6.480000,20.9900,85.99000,6783.020
shipping_cost,9426.0,12.795142,17.181203,0.490,3.192500,6.0500,13.99000,164.730
customer_id,9426.0,1738.422236,979.167197,2.000,898.000000,1750.0000,2578.75000,3403.000
product_base_margin,9354.0,0.512189,0.135229,0.350,0.380000,0.5200,0.59000,0.850
postal_code,9426.0,52446.327286,29374.597802,1001.000,29406.000000,52302.0000,78516.00000,99362.000
profit,9426.0,139.236410,998.486483,-16476.838,-74.017375,2.5676,140.24385,16332.414
quantity_ordered_new,9426.0,13.798430,15.107688,1.000,5.000000,10.0000,17.00000,170.000
sales,9426.0,949.706272,2598.019818,1.320,61.282500,203.4550,776.40250,100119.160


In [14]:
ship_region_count_flat_df = df.groupby(['ship_mode', 'region']).size().reset_index().rename(columns = {0:'count'})
ship_region_count_flat_df


,ship_mode,region,count
0,Delivery Truck,Central,386
1,Delivery Truck,East,329
2,Delivery Truck,South,242
3,Delivery Truck,West,326
4,Express Air,Central,322
5,Express Air,East,282
6,Express Air,South,241
7,Express Air,West,262
8,Regular Air,Central,2191
9,Regular Air,East,1678


Let's say we want convert this to <br>

<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>region</th>\n      <th>Central</th>\n      <th>East</th>\n      <th>South</th>\n      <th>West</th>\n    </tr>\n    <tr>\n      <th>ship_mode</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Delivery Truck</th>\n      <td>386</td>\n      <td>329</td>\n      <td>242</td>\n      <td>326</td>\n    </tr>\n    <tr>\n      <th>Express Air</th>\n      <td>322</td>\n      <td>282</td>\n      <td>241</td>\n      <td>262</td>\n    </tr>\n    <tr>\n      <th>Regular Air</th>\n      <td>2191</td>\n      <td>1678</td>\n      <td>1471</td>\n      <td>1696</td>\n    </tr>\n  </tbody>\n</table>

There are 2 ways to do it <br>
1. pd.crosstab - For single index and single column.
2. pd.pviot_table - Can work for multiple level index <br> <br>
Let's look at pivot_table first

## pd.pivot_table  - To reshape
Take the values of a column and converts it into columns. <br>
If one column takes <i>m</i> values and another takes <i>n</i> values and you have a table with <i>m x n</i> rows and 1 column, then to convert it into <i>m  rows and n columns</i> , you can use this <br>
This is similar to swaping rows and columns in <b> pivot_table in Excel </b>

In [15]:
ship_cross_region_count_df = pd.pivot_table(ship_region_count_flat_df, 
                                            index=['ship_mode'], columns=['region'])
ship_cross_region_count_df


count                  
region         Central  East South  West
ship_mode                               
Delivery Truck     386   329   242   326
Express Air        322   282   241   262
Regular Air       2191  1678  1471  1696

Can also work if you have multiple columns

In [16]:
ship_region_count_flat_df_2 = df.groupby(['ship_mode', 'region']).agg(
    {'profit':'mean', 'shipping_cost':'max'}).reset_index()
ship_region_count_flat_df_2


,ship_mode,region,profit,shipping_cost
0,Delivery Truck,Central,228.070616,164.73
1,Delivery Truck,East,234.455276,110.20
2,Delivery Truck,South,141.964085,99.00
3,Delivery Truck,West,210.231767,110.20
4,Express Air,Central,167.144535,56.20
5,Express Air,East,139.729189,69.00
6,Express Air,South,65.838272,58.66
7,Express Air,West,159.111334,69.00
8,Regular Air,Central,172.510164,69.00
9,Regular Air,East,155.558265,69.00


In [17]:
ship_cross_region_count_df_2 = pd.pivot_table(ship_region_count_flat_df_2, 
                                              index=['ship_mode'], columns=['region'], values=['profit'])
ship_cross_region_count_df_2


profit                                    
region             Central        East       South        West
ship_mode                                                     
Delivery Truck  228.070616  234.455276  141.964085  210.231767
Express Air     167.144535  139.729189   65.838272  159.111334
Regular Air     172.510164  155.558265   36.695350  118.294061

## pd.pivot_table as substitue to groupby.agg
Direct aggregations can also be done using this

In [18]:
pd.pivot_table(df, index=['ship_mode'], columns=['region'], values=['profit'], aggfunc=np.mean)


profit                                    
region             Central        East       South        West
ship_mode                                                     
Delivery Truck  228.070616  234.455276  141.964085  210.231767
Express Air     167.144535  139.729189   65.838272  159.111334
Regular Air     172.510164  155.558265   36.695350  118.294061

In [19]:
pd.pivot_table(df, index=['ship_mode'], columns=['region'], values=['profit', 'shipping_cost'], 
               aggfunc={'profit': np.sum, 'shipping_cost':np.max})

profit                                              \
region                Central           East         South           West   
ship_mode                                                                   
Delivery Truck   88035.257810   77135.785939  34355.308509   68535.556129   
Express Air      53820.540384   39403.631277  15867.023473   41687.169540   
Regular Air     377969.768872  261026.768829  53978.860438  200626.728228   

               shipping_cost                       
region               Central   East  South   West  
ship_mode                                          
Delivery Truck        164.73  110.2  99.00  110.2  
Express Air            56.20   69.0  58.66   69.0  
Regular Air            69.00   69.0  69.00   69.0

## pd.crosstab - another substitue to groupby.agg
1. Used mainly when one has to get a simple count table (and as percentages)
2. Takes series as input, so one can give series of tow different dfs too (as far as indices are same)

In [20]:
pd.crosstab(df['ship_mode'], df['region'])

region,Central,East,South,West
ship_mode,,,,
Delivery Truck,386,329,242,326
Express Air,322,282,241,262
Regular Air,2191,1678,1471,1696


In [21]:
# To get percnetage over each index
pd.crosstab(df['ship_mode'], df['region'], normalize='index')

region,Central,East,South,West
ship_mode,,,,
Delivery Truck,0.300857,0.256430,0.188620,0.254092
Express Air,0.290876,0.254743,0.217706,0.236676
Regular Air,0.311399,0.238488,0.209068,0.241046


In [22]:
# To get percnetage over each column
pd.crosstab(df['ship_mode'], df['region'], normalize='columns')

region,Central,East,South,West
ship_mode,,,,
Delivery Truck,0.133149,0.143731,0.123849,0.142732
Express Air,0.111073,0.123198,0.123337,0.114711
Regular Air,0.755778,0.733071,0.752815,0.742557


In [23]:
# To get percnetage overall
pd.crosstab(df['ship_mode'], df['region'], normalize='all')

region,Central,East,South,West
ship_mode,,,,
Delivery Truck,0.040951,0.034903,0.025674,0.034585
Express Air,0.034161,0.029917,0.025568,0.027795
Regular Air,0.232442,0.178018,0.156058,0.179928


Can do groupby aggregated functions too

In [24]:
pd.crosstab(index=df['ship_mode'], columns=df['region'], values=df['profit'], aggfunc='mean')

region,Central,East,South,West
ship_mode,,,,
Delivery Truck,228.070616,234.455276,141.964085,210.231767
Express Air,167.144535,139.729189,65.838272,159.111334
Regular Air,172.510164,155.558265,36.695350,118.294061


# Concat, Append, Merge, Join

In [25]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [26]:
df['ship_mode'].value_counts()

Regular Air       7036
Delivery Truck    1283
Express Air       1107
Name: ship_mode, dtype: int64

In [27]:
df_regular_air = df[df['ship_mode'] == 'Regular Air'].copy()
df_express_air = df[df['ship_mode'] == 'Express Air'].copy()

In [28]:
print(df_regular_air['ship_mode'].value_counts())
print()
print(df_express_air['ship_mode'].value_counts())

Regular Air    7036
Name: ship_mode, dtype: int64

Express Air    1107
Name: ship_mode, dtype: int64


## pd.concat - append or attach dataframe together

In [29]:
df_all_air = pd.concat([df_regular_air, df_express_air], axis=0)
df_all_air['ship_mode'].value_counts()


Regular Air    7036
Express Air    1107
Name: ship_mode, dtype: int64

Using axis=0, dfs can be joined veritcally.  Index should be same for this

In [30]:
df.head(2)

,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [31]:
df_1 = df[['row_id', 'order_priority']]
df_2 = df[['customer_name', 'customer_id', 'customer_segment']]
print(df_1.shape)
print(df_2.shape)

(9426, 2)
(9426, 3)


In [32]:
df_vconcat = pd.concat([df_1, df_2], axis=1)
print(df_vconcat.shape)
df_vconcat.head()

(9426, 5)


,row_id,order_priority,customer_name,customer_id,customer_segment
0,18606,Not Specified,Janice Fletcher,2,Corporate
1,20847,High,Bonnie Potter,3,Corporate
2,23086,Not Specified,Bonnie Potter,3,Corporate
3,23087,Not Specified,Bonnie Potter,3,Corporate
4,23088,Not Specified,Bonnie Potter,3,Corporate


## Merge or join

In [33]:
df_users = pd.read_csv(input_data_folder + 'Superstore_Users.csv')
print(df_users.shape)
df_users

(4, 2)


,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [34]:
region_wise_count_df = df['region'].value_counts().rename('count').reset_index()
region_wise_count_df


,index,count
0,Central,2899
1,East,2289
2,West,2284
3,South,1954


In [35]:
pd.merge(region_wise_count_df, df_users,
        how='left', left_on=['index'], right_on=['Region'])

,index,count,Region,Manager
0,Central,2899,Central,Chris
1,East,2289,East,Erin
2,West,2284,West,William
3,South,1954,South,Sam


If dfs have same column names, then left_on, right_on is not required. You can use the below

In [36]:
pd.merge(region_wise_count_df.rename(columns = {'index':'Region'}), df_users,
        how='left', on='Region')

,Region,count,Manager
0,Central,2899,Chris
1,East,2289,Erin
2,West,2284,William
3,South,1954,Sam


# Lag, shift, rolling aggregations - Groupby Additional functions
These operations comes in handy in
1. Time series opetaions
2. Computing rolling features
3. Comparing features of this row with previous/next. Without the use of loop

In [37]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [38]:
df_needed = df[['row_id', 'customer_id', 'order_date', 'quantity_ordered_new', 'sales']].copy()
df_needed.head(2)

,row_id,customer_id,order_date,quantity_ordered_new,sales
0,18606,2,2012-05-28,2,5.90
1,20847,3,2010-07-07,4,13.01


For each customer, let's try to find out mean of frequency of his orders. On an average in how many days a customer reorders. <br>
For this, we will need difference of his subsequent orders. Let's get that

In [39]:
# First sort the df by customer_id and order date
df_needed.sort_values(['customer_id', 'order_date'], ascending=True, inplace=True)
df_needed.head()

,row_id,customer_id,order_date,quantity_ordered_new,sales
0,18606,2,2012-05-28,2,5.90
1,20847,3,2010-07-07,4,13.01
2,23086,3,2011-07-27,7,49.92
3,23087,3,2011-07-27,7,41.64
4,23088,3,2011-07-27,8,1446.67


## groupby.shift()

In [40]:
df_needed['previous_order_date'] = df.groupby('customer_id')['order_date'].shift(1)
df_needed.head()

,row_id,customer_id,order_date,quantity_ordered_new,sales,previous_order_date
0,18606,2,2012-05-28,2,5.90,NaN
1,20847,3,2010-07-07,4,13.01,NaN
2,23086,3,2011-07-27,7,49.92,2010-07-07
3,23087,3,2011-07-27,7,41.64,2011-07-27
4,23088,3,2011-07-27,8,1446.67,2011-07-27


## groupby.cumsum()

In [41]:
df_needed['cumulative_quantity_ordered'] = df.groupby('customer_id')['quantity_ordered_new'].cumsum()
df_needed.tail(10)

,row_id,customer_id,order_date,quantity_ordered_new,sales,previous_order_date,cumulative_quantity_ordered
9416,24933,3400,2012-11-29,8,67.50,2012-07-04,39
9417,23413,3400,2013-04-27,8,878.71,2012-11-29,47
9418,23414,3400,2013-04-27,1,43.90,2013-04-27,48
9419,24912,3400,2013-10-04,19,596.35,2013-04-27,67
9420,18329,3402,2011-04-12,10,1969.31,NaN,10
9421,20275,3402,2013-05-14,13,447.87,2011-04-12,23
9422,20276,3402,2013-05-14,3,13.23,2013-05-14,26
9423,24491,3402,2013-09-12,4,2215.93,2013-05-14,30
9424,25914,3403,2010-02-08,5,506.50,NaN,5
9425,24492,3403,2013-09-12,23,172.48,2010-02-08,28


## S.no. for rows of particular customer_id

In [42]:
df_needed['s_no'] = df.groupby('customer_id')['quantity_ordered_new'].cumcount()
df_needed.tail(10)

,row_id,customer_id,order_date,quantity_ordered_new,sales,previous_order_date,cumulative_quantity_ordered,s_no
9416,24933,3400,2012-11-29,8,67.50,2012-07-04,39,3
9417,23413,3400,2013-04-27,8,878.71,2012-11-29,47,4
9418,23414,3400,2013-04-27,1,43.90,2013-04-27,48,5
9419,24912,3400,2013-10-04,19,596.35,2013-04-27,67,6
9420,18329,3402,2011-04-12,10,1969.31,NaN,10,0
9421,20275,3402,2013-05-14,13,447.87,2011-04-12,23,1
9422,20276,3402,2013-05-14,3,13.23,2013-05-14,26,2
9423,24491,3402,2013-09-12,4,2215.93,2013-05-14,30,3
9424,25914,3403,2010-02-08,5,506.50,NaN,5,0
9425,24492,3403,2013-09-12,23,172.48,2010-02-08,28,1


## rolling_mean()

Find mean of 2 consecutive order quantity for each customer

In [43]:
df_needed.groupby('customer_id')['quantity_ordered_new'].rolling(window=2).mean().tail(10)

customer_id      
3400         9416     5.5
             9417     8.0
             9418     4.5
             9419    10.0
3402         9420     NaN
             9421    11.5
             9422     8.0
             9423     3.5
3403         9424     NaN
             9425    14.0
Name: quantity_ordered_new, dtype: float64

Find mean of last 2 consecutive order quantity (not including self) for each customer

In [44]:
df_needed.groupby('customer_id')['quantity_ordered_new'].shift(1).rolling(window=2).mean().tail(10)

9416     8.0
9417     5.5
9418     8.0
9419     4.5
9420     NaN
9421     NaN
9422    11.5
9423     8.0
9424     NaN
9425     NaN
Name: quantity_ordered_new, dtype: float64

# Looping over dataframe rows
<b>Avoid</b> this as much as possible

In [45]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [47]:
new_df_list = []
for index, row in df.iterrows():
    # row is a series here
    if row['ship_mode'] == 'Regular Air':
        new_df_list.append(row.tolist())

new_df = pd.DataFrame(new_df_list, columns = df.columns.tolist())
print(new_df.shape)

(7036, 24)


In [48]:
# Same thing could have been doing using
df[df['ship_mode'] == 'Regular Air'].shape

(7036, 24)

# Datetime analysis

In [52]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [53]:
df_needed = df[['row_id', 'order_date']].copy()

## Convert string to datetime object

In [54]:
df_needed['order_date_datetime_object'] = pd.to_datetime(df_needed['order_date'])
df_needed[['order_date', 'order_date_datetime_object']].dtypes

order_date                            object
order_date_datetime_object    datetime64[ns]
dtype: object

## Convert datetime object to epoch

In [55]:
df_needed['order_date_epoch_milliseconds'] = df_needed['order_date_datetime_object'].apply(
    lambda x:  (x - datetime.datetime(1970,1,1)).total_seconds()*1000)
df_needed.head(2)

,row_id,order_date,order_date_datetime_object,order_date_epoch_milliseconds
0,18606,2012-05-28,2012-05-28,1.338163e+12
1,20847,2010-07-07,2010-07-07,1.278461e+12


## Convert epoch to datetime object
<b> Caution :- </b> pd.to_datetime converts to utc time, not local time

In [56]:
df_needed['order_date_datetime_object_2'] = pd.to_datetime(df_needed['order_date_epoch_milliseconds'], unit='ms')

To convert to local time, use below syntax

In [57]:
df_needed['order_date_datetime_object_3'] = df_needed['order_date_epoch_milliseconds'].apply(
    lambda x: datetime.datetime.fromtimestamp(x/1000))

In [58]:
df_needed.head(2)

,row_id,order_date,order_date_datetime_object,order_date_epoch_milliseconds,order_date_datetime_object_2,order_date_datetime_object_3
0,18606,2012-05-28,2012-05-28,1.338163e+12,2012-05-28,2012-05-28 05:30:00
1,20847,2010-07-07,2010-07-07,1.278461e+12,2010-07-07,2010-07-07 05:30:00


Not the difference of 5.30 hours in _2 and _3 as this code was originally written in Indian Standard Time

## Get date, month, week, weekday from datetime object

<b>Note</b> :- weekday gives 0 = Monday and 6 = Sunday

In [59]:
# Get date from datetime
df_needed['order_date_date_object'] = df_needed['order_date_datetime_object'].apply(lambda x: x.date())

# Get month from datetime
df_needed['order_date_month'] = df_needed['order_date_datetime_object'].apply(
    lambda x: x.strftime('%Y-m%m')) # or x.month will also work

# Get week from datetime
df_needed['order_date_week'] = df_needed['order_date_datetime_object'].apply(
    lambda x: x.strftime('%Y-w%V')) # x.week will also work

# Get weekday from datetime
df_needed['order_date_weekday'] = df_needed['order_date_datetime_object'].apply(lambda x: x.weekday())

# Get day from datetime
df_needed['order_date_day'] = df_needed['order_date_datetime_object'].apply(lambda x: x.day)

df_needed.head()

,row_id,order_date,order_date_datetime_object,order_date_epoch_milliseconds,order_date_datetime_object_2,order_date_datetime_object_3,order_date_date_object,order_date_month,order_date_week,order_date_weekday,order_date_day
0,18606,2012-05-28,2012-05-28,1.338163e+12,2012-05-28,2012-05-28 05:30:00,2012-05-28,2012-m05,2012-w22,0,28
1,20847,2010-07-07,2010-07-07,1.278461e+12,2010-07-07,2010-07-07 05:30:00,2010-07-07,2010-m07,2010-w27,2,7
2,23086,2011-07-27,2011-07-27,1.311725e+12,2011-07-27,2011-07-27 05:30:00,2011-07-27,2011-m07,2011-w30,2,27
3,23087,2011-07-27,2011-07-27,1.311725e+12,2011-07-27,2011-07-27 05:30:00,2011-07-27,2011-m07,2011-w30,2,27
4,23088,2011-07-27,2011-07-27,1.311725e+12,2011-07-27,2011-07-27 05:30:00,2011-07-27,2011-m07,2011-w30,2,27


## Iso year,week,weekday
Will be useful if week wise analysis is needed that is spreading over years. <br>
Convention is set to avoid confusion in week on year ends and year starts <br>
https://en.wikipedia.org/wiki/ISO_week_date <br>

<b>Note</b> :- isoweekday gives 1 = Monday and 7 = Sunday

In [60]:
df_needed['iso_year'] = df_needed['order_date_datetime_object'].apply(lambda x: x.isocalendar()[0])
df_needed['iso_week'] = df_needed['order_date_datetime_object'].apply(lambda x: x.isocalendar()[1])
df_needed['day_of_week'] = df_needed['order_date_datetime_object'].apply(lambda x: x.isocalendar()[2])
df_needed.head()

,row_id,order_date,order_date_datetime_object,order_date_epoch_milliseconds,order_date_datetime_object_2,order_date_datetime_object_3,order_date_date_object,order_date_month,order_date_week,order_date_weekday,order_date_day,iso_year,iso_week,day_of_week
0,18606,2012-05-28,2012-05-28,1.338163e+12,2012-05-28,2012-05-28 05:30:00,2012-05-28,2012-m05,2012-w22,0,28,2012,22,1
1,20847,2010-07-07,2010-07-07,1.278461e+12,2010-07-07,2010-07-07 05:30:00,2010-07-07,2010-m07,2010-w27,2,7,2010,27,3
2,23086,2011-07-27,2011-07-27,1.311725e+12,2011-07-27,2011-07-27 05:30:00,2011-07-27,2011-m07,2011-w30,2,27,2011,30,3
3,23087,2011-07-27,2011-07-27,1.311725e+12,2011-07-27,2011-07-27 05:30:00,2011-07-27,2011-m07,2011-w30,2,27,2011,30,3
4,23088,2011-07-27,2011-07-27,1.311725e+12,2011-07-27,2011-07-27 05:30:00,2011-07-27,2011-m07,2011-w30,2,27,2011,30,3


<b> Advice :- </b> Know the datatypes of datetime columns. datetime, int, float, string (also known as object)

In [61]:
df_needed.dtypes

row_id                                    int64
order_date                               object
order_date_datetime_object       datetime64[ns]
order_date_epoch_milliseconds           float64
order_date_datetime_object_2     datetime64[ns]
order_date_datetime_object_3     datetime64[ns]
order_date_date_object                   object
order_date_month                         object
order_date_week                          object
order_date_weekday                        int64
order_date_day                            int64
iso_year                                  int64
iso_week                                  int64
day_of_week                               int64
dtype: object

# pandas options - To better see tabular data

In [62]:
df = pd.read_csv(input_data_folder + 'Superstore_Orders.csv')
df.columns = [ '_'.join(col.lower().split(' ')) for col in df.columns]
print(df.shape)
df.head(2)

(9426, 24)


,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,...,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


Notice that we aer not able to see all the rows here. We can change that by setting max_rows option of pandas. <br>
Similarly, there are max_columns and other options which make seeing the data easy <br>
Some of them which I use are listed below

In [63]:
pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [64]:
df.head(2)

,row_id,order_priority,discount,unit_price,shipping_cost,customer_id,customer_name,ship_mode,customer_segment,product_category,product_sub-category,product_container,product_name,product_base_margin,region,state_or_province,city,postal_code,order_date,ship_date,profit,quantity_ordered_new,sales,order_id
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,Small Box,Avery 49,0.36,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,0.54,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


# Jupyter nbextensions

If using jupyter, do install extension of jupyter. It allows 
1. Exucustion time of each cell.
2. Code folding. 
3. Table of contents
4. Other more things which I have not used

To install, run pip install jupyter_contrib_nbextensions <br>
https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html

# Extra
A lot is there apart from these. Please explore.